In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open(r'C:\Users\nbeac\.secret\yelp_api.json') as f:
    login = json.load(f)
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# Set the API call parameters 
LOCATION = 'Reseda, CA, 91335'
TERM = 'Tacos'

In [4]:
# Create our JSON_FILE
JSON_FILE = f"Data/results_in_progress_Tacos.json"
JSON_FILE

'Data/results_in_progress_Tacos.json'

In [6]:
# Check if JSON_FILE exists with for-loop
file_exists = os.path.isfile(JSON_FILE)

# if it does not exist:
if file_exists == False:
    
    ## Create any needed folders
    ## Get folder name only
    folder = os.path.dirname(JSON_FILE)
    
    ## if JSON_FILE includes a folder:
    if len(folder)>0:
        os.makedirs(folder,exist_ok=True)
    
    ## Inform User and save empty list
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
        json.dump([],f)
    
    ## if it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists!")

## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

[i] Data/results_in_progress_Tacos.json already exists!
- 0 previous results found.


In [8]:
results = yelp_api.search_query(location=LOCATION,
                               term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
total_results = results['total']
total_results

296

In [10]:
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
n_pages = math.ceil((results['total']-n_results)/results_per_page)
n_pages

15

In [12]:
for i in tqdm_notebook(range(n_pages)):
    time.sleep(.2)

  0%|          | 0/15 [00:00<?, ?it/s]

In [14]:
for i in tqdm_notebook(range(1, n_pages+1)):
    time.sleep(.2)
    
    with open(JSON_FILE,'r') as f:
        previous_results = json.load(f)
    
    n_results = len(previous_results)
    
    results = yelp_api.search_query(location = LOCATION,
                                   term = TERM,
                                   offset = n_results)
    
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/15 [00:00<?, ?it/s]

In [15]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,pudwsHTaasbvqrB00UYA_w,el-tijuanense-los-angeles-2,El TIjuanense,https://s3-media1.fl.yelpcdn.com/bphoto/GTg8q6...,False,https://www.yelp.com/biz/el-tijuanense-los-ang...,130,"[{'alias': 'tacos', 'title': 'Tacos'}]",4.0,"{'latitude': 34.23172, 'longitude': -118.5621}","[delivery, pickup]","{'address1': '8870 Corbin Ave', 'address2': No...",+18186268186,(818) 626-8186,3477.459232,NaN
1,n0ImmQ9RjWulUYtvGeyzaw,joeys-tacos-reseda,Joey's Tacos,https://s3-media3.fl.yelpcdn.com/bphoto/bCTZPD...,False,https://www.yelp.com/biz/joeys-tacos-reseda?ad...,167,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 34.2017811919696, 'longitude': -1...",[delivery],"{'address1': '7215 Reseda Blvd', 'address2': '...",+18185785220,(818) 578-5220,777.520188,$
2,bb7DrutC1qvifYd3a-ykdQ,tacos-al-gusto-reseda-4,Tacos al Gusto,https://s3-media2.fl.yelpcdn.com/bphoto/McXaZM...,False,https://www.yelp.com/biz/tacos-al-gusto-reseda...,184,"[{'alias': 'catering', 'title': 'Caterers'}, {...",5.0,"{'latitude': 34.20367396652554, 'longitude': -...",[],"{'address1': None, 'address2': None, 'address3...",+18184386690,(818) 438-6690,3.749069,$$
3,uLK-xcfGGKKCrkOJoBAc9w,el-tacote-los-angeles-3,El Tacote,https://s3-media1.fl.yelpcdn.com/bphoto/PKadPd...,False,https://www.yelp.com/biz/el-tacote-los-angeles...,81,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",4.5,"{'latitude': 34.236773, 'longitude': -118.53659}","[delivery, pickup]","{'address1': '9167 Reseda Blvd', 'address2': N...",+18189932421,(818) 993-2421,3758.787506,$
4,z0pGYc_VkgJuw9uZrGvMhQ,el-tijuanense-los-angeles,El Tijuanense,https://s3-media2.fl.yelpcdn.com/bphoto/IRL7Oz...,False,https://www.yelp.com/biz/el-tijuanense-los-ang...,115,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",4.5,"{'latitude': 34.200404, 'longitude': -118.508698}","[delivery, pickup]","{'address1': '7140 Louise Ave', 'address2': No...",+18183212838,(818) 321-2838,3329.979013,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
291,8SCbD-v3IpEXZMg92wfAhQ,burger-king-northridge-4,Burger King,https://s3-media3.fl.yelpcdn.com/bphoto/tAJsr_...,False,https://www.yelp.com/biz/burger-king-northridg...,61,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.0,"{'latitude': 34.23506701375673, 'longitude': -...",[delivery],"{'address1': '19640 Nordhoff St', 'address2': ...",+18183418466,(818) 341-8466,3808.666115,$
292,veBOS6PAc557HCVmyeH9fw,pickwick-pub-woodland-hills,Pickwick Pub,https://s3-media3.fl.yelpcdn.com/bphoto/BzCeC6...,False,https://www.yelp.com/biz/pickwick-pub-woodland...,315,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.0,"{'latitude': 34.16652099548, 'longitude': -118...","[pickup, delivery]","{'address1': '21010 Ventura Blvd', 'address2':...",+18183409673,(818) 340-9673,5955.819857,$$
293,QE4ttw5U8rVH77x3-PDNBA,wienerschnitzel-canoga-park,Wienerschnitzel,https://s3-media2.fl.yelpcdn.com/bphoto/2jVaax...,False,https://www.yelp.com/biz/wienerschnitzel-canog...,57,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.0,"{'latitude': 34.2013405634972, 'longitude': -1...","[pickup, delivery]","{'address1': '20925 Sherman Way', 'address2': ...",+18185872978,(818) 587-2978,4121.086124,$
294,S1K4Ru1R6IzWxbdsjsHYyA,california-pizza-kitchen-tarzana-2,California Pizza Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/uMc37Q...,False,https://www.yelp.com/biz/california-pizza-kitc...,452,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 34.171389, 'longitude': -118.543352}",[delivery],"{'address1': '18800 Ventura Blvd', 'address2':...",+18183459925,(818) 345-9925,3595.823626,$$
295,JYS8vmR1sLZcZclsiYG-ig,california-pizza-kitchen-northridge-3,California Pizza Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/NGqJ9t...,False,https://www.yelp.com/biz/california-pizza-kitc...,657,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 34.2402165214109, 'longitude': -1...",[delivery],"{'address1': '9301 Tampa Ave', 'address2': 'Un...",+18187015035,(818) 701-5035,4175.648005,$$


In [16]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

0

In [17]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_crab_cakes.csv.gz', compression='gzip', index=False)